# Process data

### Important

In order to process the airport, flight, and weather data, you must already have it available. If you have not already acquired the data, open notebook `01_get_data` and run all its cells.

In [1]:
import os
import pandas as pd
import re  # regular expressions

In [2]:
# Make `resources` the working directory, and set `data_dir`
os.chdir(os.path.join('..','resources'))
data_dir = os.path.join('.','data')

In [3]:
os.listdir(data_dir)

['05-2019.csv',
 '06-2019.csv',
 '07-2019.csv',
 '08-2019.csv',
 '09-2019.csv',
 '10-2019.csv',
 '11-2019.csv',
 '12-2019.csv',
 'GlobalAirportDatabase.txt',
 'GlobalAirportDatabase.zip',
 'historical-flight-and-weather-data.zip',
 'readme.txt']

In [4]:
faw_source_files = list(filter(
                            lambda item: re.fullmatch(
                                '\d{2}\-2019\.csv',
                                item,
                                flags=re.I
                            ) is not None,
                            os.listdir(data_dir)
                        ))

# secondary_source_file = 'gadb_postgresql_create_airports_table.sql'
secondary_source_file = 'GlobalAirportDatabase.txt'

In [5]:
def print_shape(df):
    df_rows, df_cols = df.shape
    print(f"{df_rows:,} rows × {df_cols:,} columns")

## Flights and Weather Data Set

In [6]:
faw_df = pd.concat([
    pd.read_csv(os.path.join(data_dir,filename))
    for filename in faw_source_files
])

print_shape(faw_df)

5,512,903 rows × 35 columns


In [7]:
faw_df.head()

,carrier_code,flight_number,origin_airport,destination_airport,date,scheduled_elapsed_time,tail_number,departure_delay,arrival_delay,delay_carrier,...,HourlyPrecipitation_x,HourlyStationPressure_x,HourlyVisibility_x,HourlyWindSpeed_x,STATION_y,HourlyDryBulbTemperature_y,HourlyPrecipitation_y,HourlyStationPressure_y,HourlyVisibility_y,HourlyWindSpeed_y
0,AS,121,SEA,ANC,2019-05-01,215,N615AS,-8,-16,0,...,0.0,29.59,10.0,8.0,7.027253e+10,42.0,0.0,30.16,10.0,3.0
1,F9,402,LAX,DEN,2019-05-01,147,N701FR,17,-4,0,...,0.0,29.65,10.0,3.0,7.256500e+10,34.0,0.0,24.43,4.0,0.0
2,F9,662,SFO,DEN,2019-05-01,158,N346FR,44,27,0,...,0.0,29.98,10.0,6.0,7.256500e+10,34.0,0.0,24.43,4.0,0.0
3,F9,790,PDX,DEN,2019-05-01,156,N332FR,24,10,0,...,0.0,29.98,10.0,0.0,7.256500e+10,34.0,0.0,24.43,4.0,0.0
4,AS,108,ANC,SEA,2019-05-01,210,N548AS,-9,-31,0,...,0.0,30.18,10.0,5.0,7.279302e+10,44.0,0.0,29.58,10.0,7.0


In [8]:
# Get data types and number of null values for each column
pd.concat(
    [
        faw_df.dtypes,
        faw_df.isna().sum()
    ],
    axis=1,
    keys=['data_type','null_count']
)

,data_type,null_count
carrier_code,object,0
flight_number,int64,0
origin_airport,object,0
destination_airport,object,0
date,object,0
scheduled_elapsed_time,int64,0
tail_number,object,13556
departure_delay,int64,0
arrival_delay,int64,0
delay_carrier,int64,0


**Note:** Eventually, `cancelled_code` will be our target column for a machine-learning algorithm.  
Because the column in the source data is `cancelled_code` and not `canceled_code`, the double-l spelling will be used in this work.

In [9]:
# Examine the `carrier_code` column
faw_df.carrier_code.value_counts()

AA    1438798
DL    1207720
UA    1070050
WN     918320
AS     304198
B6     199314
NK     142041
F9      97482
G4      71728
HA      63252
Name: carrier_code, dtype: int64

### What do the codes mean?

According to the United States Department of Transportation Bureau of Transportation Statistics Airlines and Airports data, [Airline Codes](https://www.bts.gov/topics/airlines-and-airports/airline-codes) document:

**AIRLINE CODES:**
- `AA`-American Airlines Inc.
- `AS`-Alaska Airlines Inc.
- `B6`-JetBlue Airways
- `DL`-Delta Air Lines Inc.
- `F9`-Frontier Airlines Inc.
- `G4`-Allegiant Air
- `HA`-Hawaiian Airlines Inc.
- `NK`-Spirit Air Lines
- `UA`-United Air Lines Inc.
- `WN`-Southwest Airlines Co.

In [10]:
# Examine the `flight_number` column
# Are they unique?
faw_df.flight_number.duplicated(keep=False).sum()

5512864

They very much are not unique.

In [11]:
# What about the combination of `carrier_code` and `flight_number`?
faw_df[['carrier_code','flight_number']].duplicated(keep=False).sum()

5512259

Also not unique.

In [12]:
# In order to prevent the `date` column from being confused with the `date` data
# type or any `date` functions, rename the `date` column to `flight_date`
faw_df.rename(columns={'date':'flight_date'}, errors='ignore', inplace=True)

In [13]:
# Combine `year`, `month`, and `day` into a single `string` in the same format as the `date` column
# and check for equality against the actual date column.

# Check only a few rows
(
    faw_df[['year','month','day']][:3]
    .apply(lambda row: '-'.join([val.zfill(2) for val in row.values.astype(str)]), axis=1)
    .equals(
        faw_df.flight_date[:3]
    )
)

# Check all the rows
# (
#     faw_df[['year','month','day']]
#     .apply(lambda row: '-'.join([val.zfill(2) for val in row.values.astype(str)]), axis=1)
#     .equals(
#         faw_df.flight_date
#     )
# )

True

**Note:** The all-rows check, above, is commented out because it takes a long time, but when run, it does show equality between the entire `flight_date` series and the combined `year`-`month`-`day` series.

Because `year`, `month`, and `day` were originally stored as `int64` values, this also tells us that all the values in `flight_date` are properly formatted (no leading or trailing spaces, *etc*.).

The data is therefore redundant, and we don't need both.

`weekday` is likewise redundant, since it can be calculated from `flight_date`.

In [14]:
# Drop redundant date columns
faw_df.drop(
    columns=['year','month','day','weekday'],
    errors='ignore',
    inplace=True
)

print_shape(faw_df)

5,512,903 rows × 31 columns


In [15]:
# Examine `cancelled_code` column
faw_df.cancelled_code.value_counts()

N    5426150
B      41919
A      23451
C      21370
D         13
Name: cancelled_code, dtype: int64

### What do the codes mean?

According to the United States Department of Transportation Bureau of Transportation Statistics Airlines and Airports data, [Number 14 - On-Time Reporting](https://www.bts.gov/topics/airlines-and-airports/number-14-time-reporting):

**CANCELLATION CODES**
- `A`-Carrier Caused
- `B`-Weather
- `C`-National Aviation System
- `D`-Security

\[`N` is not on the list and represents "None" or "Not cancelled".\]

We are only interested in flights that were cancelled due to weather, so we will keep only rows with `cancelled_code` `B` or `N`.

In [16]:
faw_df = faw_df.loc[faw_df.cancelled_code.isin(['B','N'])]

print_shape(faw_df)

5,468,069 rows × 31 columns


In [17]:
# Check that there are now only `B` and `N` values
faw_df.cancelled_code.value_counts()

N    5426150
B      41919
Name: cancelled_code, dtype: int64

In [18]:
# Convert `cancelled_code` column into boolean `cancelled` column, where
# `B` = True (*was* cancelled) and `N` = False (*was not* cancelled)

try:
    print("Converting cancelled_code column to boolean… ", end="")
    faw_df.cancelled_code = (faw_df.cancelled_code == 'B')
    faw_df.rename(columns={'cancelled_code':'cancelled'},inplace=True)
    print()
except AttributeError:
    print("Column has already been processed.")

faw_df.cancelled.value_counts()

Converting cancelled_code column to boolean… 


False    5426150
True       41919
Name: cancelled, dtype: int64

In [19]:
# How many flights were cancelled|not cancelled vs. how many departed|arrived

departed = ~faw_df.actual_departure_dt.isna()
arrived = ~faw_df.actual_arrival_dt.isna()

mult_ix = pd.MultiIndex.from_tuples([
    ('departed',True),
    ('departed',False),
    ('arrived',True),
    ('arrived',False),
])

mult_cols = pd.MultiIndex.from_tuples([
    ('cancelled',False),
    ('cancelled',True)
])

pd.DataFrame(
    data=[
        [
            faw_df.loc[(~faw_df.cancelled) & (departed)].shape[0],
            faw_df.loc[(faw_df.cancelled) & (departed)].shape[0]
        ],
        [
            faw_df.loc[(~faw_df.cancelled) & (~departed)].shape[0],
            faw_df.loc[(faw_df.cancelled) & (~departed)].shape[0]
        ],
        [
            faw_df.loc[(~faw_df.cancelled) & (arrived)].shape[0],
            faw_df.loc[(faw_df.cancelled) & (arrived)].shape[0]
        ],
        [
            faw_df.loc[(~faw_df.cancelled) & (~arrived)].shape[0],
            faw_df.loc[(faw_df.cancelled) & (~arrived)].shape[0]
        ]
    ],
    index=mult_ix,
    columns=mult_cols
)

cancelled       
                   False   True
departed True    5426150   1854
         False         0  40065
arrived  True    5424261      0
         False      1889  41919

In [20]:
# Does anything stand out for cancelled flights that still departed?
faw_df.loc[faw_df.cancelled & departed].head().transpose()

,16715,17002,17815,18640,18750
carrier_code,AA,AA,AA,AA,AA
flight_number,1393,346,2761,1271,5821
origin_airport,OKC,DFW,DFW,IAH,DFW
destination_airport,DFW,MSY,STL,DFW,ELP
flight_date,2019-05-01,2019-05-01,2019-05-01,2019-05-01,2019-05-01
scheduled_elapsed_time,69,85,105,75,104
tail_number,N751UW,N357PV,N971TW,N898NN,N243LR
departure_delay,176,83,111,113,28
arrival_delay,0,0,0,0,0
delay_carrier,0,0,0,0,0


In [21]:
# What about non-cancelled flights that didn't arrive?
faw_df.loc[~faw_df.cancelled & ~arrived].head().transpose()

,5154,12535,13657,16277,17368
carrier_code,AS,AA,WN,WN,UA
flight_number,55,2028,2272,2212,6296
origin_airport,SCC,MEM,PDX,ABQ,IAD
destination_airport,BRW,DFW,DAL,DAL,DFW
flight_date,2019-05-01,2019-05-01,2019-05-01,2019-05-01,2019-05-01
scheduled_elapsed_time,45,99,230,105,209
tail_number,N609AS,N749US,N931WN,N788SA,N87353
departure_delay,29,398,-2,-5,212
arrival_delay,0,0,0,0,0
delay_carrier,0,0,0,0,0


### Establish `flights_and_weather` table definition

Steps:
1. Translate Pandas (*i.e.*, Numpy) data types into (default) PostgreSQL data types.
2. Edit data types on specific columns (especially those of type `object`) taking into consideration:
   - the length of data for any `char` or `varchar` columns
      - If values are always the same length, use `char`; if they vary, use `varchar`.
   - <s>`UNIQUE` constraints</s>
      - \[We already examined columns that might potentially have a `UNIQUE` constraint and determined that none do.\]
   - `NOT NULL` constraints
   - `FOREIGN KEY` columns that will `REFERENCE` a column from the `airports` table
6. Generate a CREATE TABLE string.

In [22]:
# Dictionary to translate Numpy data types into default PostgreSQL data types
type_repl = {
    'bool':'boolean',
    'float64':'numeric',
    'int64':'integer',
    'object':'text'
}

In [23]:
col_defs = pd.DataFrame(
    columns=['column','data_type'],
    data=zip(faw_df.dtypes.index, map(type_repl.get, [dt.name for dt in faw_df.dtypes.values]))
)

col_defs.head()

,column,data_type
0,carrier_code,text
1,flight_number,integer
2,origin_airport,text
3,destination_airport,text
4,flight_date,text


In [24]:
# Examine columns of type `object`
faw_df.select_dtypes('object').columns.tolist()

['carrier_code',
 'origin_airport',
 'destination_airport',
 'flight_date',
 'tail_number',
 'scheduled_departure_dt',
 'scheduled_arrival_dt',
 'actual_departure_dt',
 'actual_arrival_dt']

- `carrier_code`, `origin_airport`, `destination_airport`, and `tail_number` should have either `char` or `varchar` types (depending on the lengths of their respective values).
  - Although we will be uploading `carrier_code` and `tail_number` into the database, they are for identification purposes, only; they will not be features for the machine-learning model to come later.
  - `origin_airport` and `destination_airport` will serve as `FOREIGN KEY`s that will `REFERENCE` the `iata_code` column from the `airports` table. IATA Codes are strictly three letters, and so the columns should have `char(3)` data types.
- `flight_date` should have a `date` data type.
- All columns ending in `_dt` should have `datetime` data types.
  - Similar to some previously mentioned columns, whle `actual_departure_dt` and `actual_arrival_dt` *can* be stored in the database, they absolutely should ***not*** be used as features for the machine learning model, because their presence or absence *defines* what it means for a flight to be cancelled, which is exactly what the model is intended to predict.

In [25]:
# Function to print the shortest and longest values in a column
def min_max_column_length(test_column):
    col_lengths = [len(val) for val in faw_df[test_column].dropna()]
    print(f"{test_column} length: ({min(col_lengths)},{max(col_lengths)})")

In [26]:
min_max_column_length('carrier_code')
min_max_column_length('tail_number')

carrier_code length: (2,2)
tail_number length: (5,6)


In [27]:
change_list = {
    'carrier_code':'char(2)',
    'origin_airport':'char(3)',
    'destination_airport':'char(3)',
    'flight_date':'date',
    'tail_number':'varchar(6)'
}

for col in change_list.keys():
    col_defs.data_type.loc[col_defs.column == col] = change_list[col]

In [28]:
col_defs.data_type.loc[col_defs.column.str.contains('_dt')] = 'timestamp'

In [29]:
# Set columns to `NOT NULL` if the columns…
# 1) have no missing values in the source data
# 2) do not have names ending in '_airport' (those will be handled later as FOREIGN KEYS)
# 3) are not already designated as `NOT NULL`

nn = (
    (faw_df.isna().sum().values == 0)
    &
    ~col_defs.column.str.contains('_airport')
    &
    ~col_defs.data_type.str.contains(' NOT NULL')
)

col_defs.data_type.loc[nn] = (col_defs.data_type.loc[nn] + ' NOT NULL')

In [30]:
# Similar rules for FOREIGN KEY columns

fk = (col_defs.column.str.contains('_airport') & ~col_defs.data_type.str.contains(' NOT NULL'))

col_defs.data_type.loc[fk] = (col_defs.data_type.loc[fk] + ' NOT NULL REFERENCES airports (iata_code)')

In [31]:
col_defs

,column,data_type
0,carrier_code,char(2) NOT NULL
1,flight_number,integer NOT NULL
2,origin_airport,char(3) NOT NULL REFERENCES airports (iata_code)
3,destination_airport,char(3) NOT NULL REFERENCES airports (iata_code)
4,flight_date,date NOT NULL
5,scheduled_elapsed_time,integer NOT NULL
6,tail_number,varchar(6)
7,departure_delay,integer NOT NULL
8,arrival_delay,integer NOT NULL
9,delay_carrier,integer NOT NULL


Create the `flights_and_weather` table-creation string by:
1. concatenating `col_def` values across rows (joined by ` `)
2. concatenating those rows (joined by `,\n    `)
3. inserting the result in between the appropriate table-creation text

In [32]:
faw_create = (
    'CREATE TABLE IF NOT EXISTS flights_and_weather (\n    '
    + col_defs.apply(lambda x: ' '.join(x), axis=1).str.cat(sep=',\n    ')
    + '\n);'
)

print(faw_create)

CREATE TABLE IF NOT EXISTS flights_and_weather (
    carrier_code char(2) NOT NULL,
    flight_number integer NOT NULL,
    origin_airport char(3) NOT NULL REFERENCES airports (iata_code),
    destination_airport char(3) NOT NULL REFERENCES airports (iata_code),
    flight_date date NOT NULL,
    scheduled_elapsed_time integer NOT NULL,
    tail_number varchar(6),
    departure_delay integer NOT NULL,
    arrival_delay integer NOT NULL,
    delay_carrier integer NOT NULL,
    delay_weather integer NOT NULL,
    delay_national_aviation_system integer NOT NULL,
    delay_security integer NOT NULL,
    delay_late_aircarft_arrival integer NOT NULL,
    cancelled boolean NOT NULL,
    scheduled_departure_dt timestamp NOT NULL,
    scheduled_arrival_dt timestamp NOT NULL,
    actual_departure_dt timestamp,
    actual_arrival_dt timestamp,
    STATION_x numeric,
    HourlyDryBulbTemperature_x numeric,
    HourlyPrecipitation_x numeric,
    HourlyStationPressure_x numeric,
    HourlyVisibility

## Create Database Tables

In order to connect to the database, first, make sure you have a local (running) database with the `hostname`, `database` name, `username`, and `port` number as specified in `/resources/config/gadb_pg_config.py`.

**Note:** because the `flights_and_weather` table has columns that reference columns from the `airports` table, the `airports` table must be created *before* the `flights_and_weather` table. As such, you must run `02_prepare_airport_data` before continuing in this file.

In [33]:
# Database configuration details
from config import gadb_pg_config as cfg

# To connect to SQL database
import sqlalchemy as db
# from sqlalchemy import create_engine, MetaData, Table, text, types

# To enter passwords without exposing them
from getpass import getpass

Most of the database information is in `cfg` (above). However, you will have to enter your password below.

In [34]:
password = getpass('Enter database password')

Enter database password········


In [35]:
db_string = f"postgresql+psycopg2://{cfg.username}:{password}@{cfg.hostname}:{cfg.port}/{cfg.database}"

In [36]:
engine = db.create_engine(
    future=True,
#     echo=True,
    url=db_string
)

In [37]:
# Create the `airports` and `flights_and_weather` database tables
with engine.begin() as conn:    
    conn.execute(db.text(faw_create))
print('Done.')

Done.


#### Recommendation

Check the database (via pgAdmin or some other means) to make sure that the tables exist as expected.

## Upload Data

In [38]:
db_meta = db.MetaData()

In [39]:
faw_table = db.Table('flights_and_weather', db_meta, autoload_with=engine)

### Upload Flights and Weather Data

In [40]:
# Check to see if `faw_table` has the same number of columns as `faw_df`
len(faw_table.columns.keys()) == faw_df.columns.size

True

In [41]:
# Verify `faw_table` column definitions
faw_table.columns.values()

[Column('carrier_code', CHAR(length=2), table=<flights_and_weather>, nullable=False),
 Column('flight_number', INTEGER(), table=<flights_and_weather>, nullable=False),
 Column('origin_airport', CHAR(length=3), ForeignKey('airports.iata_code'), table=<flights_and_weather>, nullable=False),
 Column('destination_airport', CHAR(length=3), ForeignKey('airports.iata_code'), table=<flights_and_weather>, nullable=False),
 Column('flight_date', DATE(), table=<flights_and_weather>, nullable=False),
 Column('scheduled_elapsed_time', INTEGER(), table=<flights_and_weather>, nullable=False),
 Column('tail_number', VARCHAR(length=6), table=<flights_and_weather>),
 Column('departure_delay', INTEGER(), table=<flights_and_weather>, nullable=False),
 Column('arrival_delay', INTEGER(), table=<flights_and_weather>, nullable=False),
 Column('delay_carrier', INTEGER(), table=<flights_and_weather>, nullable=False),
 Column('delay_weather', INTEGER(), table=<flights_and_weather>, nullable=False),
 Column('dela

In [42]:
print(db.insert(faw_table))

INSERT INTO flights_and_weather (carrier_code, flight_number, origin_airport, destination_airport, flight_date, scheduled_elapsed_time, tail_number, departure_delay, arrival_delay, delay_carrier, delay_weather, delay_national_aviation_system, delay_security, delay_late_aircarft_arrival, cancelled, scheduled_departure_dt, scheduled_arrival_dt, actual_departure_dt, actual_arrival_dt, station_x, hourlydrybulbtemperature_x, hourlyprecipitation_x, hourlystationpressure_x, hourlyvisibility_x, hourlywindspeed_x, station_y, hourlydrybulbtemperature_y, hourlyprecipitation_y, hourlystationpressure_y, hourlyvisibility_y, hourlywindspeed_y) VALUES (:carrier_code, :flight_number, :origin_airport, :destination_airport, :flight_date, :scheduled_elapsed_time, :tail_number, :departure_delay, :arrival_delay, :delay_carrier, :delay_weather, :delay_national_aviation_system, :delay_security, :delay_late_aircarft_arrival, :cancelled, :scheduled_departure_dt, :scheduled_arrival_dt, :actual_departure_dt, :act

In [43]:
# Check query to test whether `flights_and_weather` table has any rows
print(db.exists().select_from(faw_table).select())

SELECT EXISTS (SELECT * 
FROM flights_and_weather) AS anon_1


In [ ]:
# Upload
with engine.begin() as conn:
    faw_is_empty = not conn.execute(db.exists().select_from(faw_table).select()).scalar()

if (faw_is_empty):
    faw_df.to_sql(
        name='flights_and_weather',
        con=engine,
        if_exists='append',
        index=False,
        method='multi'
    )
    print('Done.')
else:
    print('`flights_and_weather` table already populated.')